In [104]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [105]:
sample_1 = pd.read_excel('/vast/amr10211/court_sample_1.xlsx')
sample_2 = pd.read_excel('/vast/amr10211/court_sample_2.xlsx')

In [106]:
# Concatenate the DataFrames vertically
concatenated_df = pd.concat([sample_1[['id', 'decision_direction', 'topic']],
                             sample_2[['id', 'decision_direction', 'topic']]])

# Remove duplicates based on the specified columns
hand_coded_df = concatenated_df.drop_duplicates(subset=['id'], keep='last')
hand_coded_df = hand_coded_df[hand_coded_df['decision_direction'].notnull()]
hand_coded_df['decision_direction'] = hand_coded_df['decision_direction'].replace({'liberal': 0, 'conservative': 1})
hand_coded_df[['topic_1', 'topic_2', 'topic_3']] = hand_coded_df['topic'].str.split('; ', expand=True)


In [107]:
label_mapping= {
    'criminal procedure': 0,
    'civil rights': 1,
    'first amendment': 2,
    'due process': 3,
    'privacy': 4,
    'attorneys': 5,
    'unions': 6,
    'economic activity': 7,
    'judicial power': 8,
    'federalism': 9,
    'interstate relations': 10,
    'federal taxation': 11,
    'miscellaneous': 12,
    'private action': 13
}

# Map using label_mapping
hand_coded_df['topic_code'] = hand_coded_df['topic'].str.lower().map(label_mapping)
hand_coded_df['topic_code1'] = hand_coded_df['topic_1'].str.lower().map(label_mapping)
hand_coded_df['topic_code2'] = hand_coded_df['topic_2'].str.lower().map(label_mapping)
hand_coded_df['topic_code3'] = hand_coded_df['topic_3'].str.lower().map(label_mapping)


In [18]:
data_df = pd.read_csv('/vast/amr10211/inference_partisan_topic_results3.csv')

/state/partition1/job-40741321/ipykernel_3957766/2965423125.py:1: DtypeWarning: Columns (17,19,20,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv('/vast/amr10211/inference_partisan_topic_results3.csv')


In [118]:
data_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,resource_uri,opinion_id,absolute_url,cluster_id,cluster,author_id,author,joined_by,date_created,date_modified,author_str,per_curiam,joined_by_str,type,sha1,page_count,download_url,local_path,extracted_by_ocr,opinions_cited,judges,judge,date_filed,token_count,predicted_label,probability,docket_id,id,court_id,imputed_author_str,imputed_person_id,imputed_court_id,unique_person_id,year_filed,appointer_id,date_start,date_termination,president_id_x,president_name,imputed_president_id,imputed_president_name,president_id_y,partisanship,predicted_topic_label,probability_class_0,probability_class_1,probability_class_2,probability_class_3,probability_class_4,probability_class_5,probability_class_6,probability_class_7,probability_class_8,probability_class_9,probability_class_10,probability_class_11,probability_class_12,probability_class_13
0,0,0,0,NaN,7347153,NaN,7427349,NaN,NaN,NaN,NaN,2022-07-26 01:06:24.86499+00,2022-07-26 01:06:24.865016+00,Clement,f,NaN,020lead,NaN,NaN,NaN,NaN,t,NaN,Clement,Clement,1954-04-08,2240,0,0.826594,64418478,64418478.0,calappdeptsuper,Clement,NaN,calappdeptsuper,Clement_calappdeptsuper,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0.024646,0.011900,0.005133,0.035044,0.001689,0.002366,0.001478,0.162833,0.745173,0.004383,0.000469,0.003317,0.000910,0.000658
1,1,1,1,NaN,7347154,NaN,7427350,NaN,NaN,NaN,NaN,2022-07-26 01:06:24.916646+00,2022-07-26 01:06:24.916666+00,Hilliard,f,NaN,020lead,NaN,NaN,NaN,NaN,t,NaN,Hilliard,Hilliard,1954-04-12,729,1,0.652668,64418479,64418479.0,calappdeptsuper,Hilliard,3908.0,calappdeptsuper,3908,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.001578,0.002278,0.000262,0.002631,0.000370,0.000832,0.000720,0.930746,0.055556,0.003017,0.000192,0.001132,0.000319,0.000366
2,2,2,2,NaN,7347207,NaN,7427399,NaN,NaN,NaN,NaN,2022-07-26 01:06:32.636426+00,2022-07-26 01:06:32.636445+00,Bishop,f,NaN,020lead,NaN,NaN,NaN,NaN,t,NaN,Bishop,Bishop,1957-10-07,856,0,0.595087,64418528,64418528.0,calappdeptsuper,Bishop,NaN,calappdeptsuper,Bishop_calappdeptsuper,1957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.001714,0.002202,0.000960,0.028589,0.000992,0.001149,0.002186,0.946257,0.009434,0.003496,0.000446,0.000644,0.000928,0.001002
3,3,3,3,NaN,7347208,NaN,7427400,NaN,NaN,NaN,NaN,2022-07-26 01:06:32.696945+00,2022-07-26 01:06:32.696958+00,Bishop,f,NaN,020lead,NaN,NaN,NaN,NaN,t,NaN,"Bishop, Shaw",Bishop,1957-10-14,1377,0,0.909268,64418529,64418529.0,calappdeptsuper,Bishop,NaN,calappdeptsuper,Bishop_calappdeptsuper,1957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.433956,0.054032,0.013417,0.025124,0.002472,0.002268,0.039595,0.307070,0.103977,0.010546,0.000842,0.002980,0.002046,0.001674
4,4,4,4,NaN,7347238,NaN,7427427,NaN,NaN,NaN,NaN,2022-07-26 01:06:36.121052+00,2022-07-26 01:06:36.121069+00,Quayle,f,NaN,020lead,NaN,NaN,NaN,NaN,t,NaN,Quayle,Quayle,1962-09-14,2833,0,0.764116,64418556,64418556.0,calappdeptsuper,Quayle,NaN,calappdeptsuper,Quayle_calappdeptsuper,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.027252,0.019230,0.206528,0.038250,0.006391,0.087428,0.008974,0.516824,0.062525,0.008942,0.003153,0.007538,0.003537,0.003428


In [119]:
inference_df = data_df[['opinion_id','predicted_label','predicted_topic_label', 'probability']]

In [120]:
merged_df = hand_coded_df.merge(inference_df, how='left', left_on = 'id', right_on = 'opinion_id')

In [121]:
merged_df['correct_decision_direction']=(merged_df['decision_direction']==merged_df['predicted_label']).astype(int)
merged_df['correct_topic_area'] = np.where(
    (merged_df['topic_code'] == merged_df['predicted_topic_label']) |
    (merged_df['topic_code1'] == merged_df['predicted_topic_label']) |
    (merged_df['topic_code2'] == merged_df['predicted_topic_label']) |
    (merged_df['topic_code3'] == merged_df['predicted_topic_label']),
    1,  # Value when the condition is true
    0   # Value when the condition is false
)

In [135]:
merged_df

,id,decision_direction,topic,topic_1,topic_2,topic_3,topic_code,topic_code1,topic_code2,topic_code3,opinion_id,predicted_label,predicted_topic_label,probability,correct_decision_direction,correct_topic_area
0,7347153,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347153,0,8,0.826594,1,0
1,7347154,neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347154,1,7,0.652668,0,0
2,7347207,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347207,0,7,0.595087,1,0
3,7347208,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347208,0,0,0.909268,1,0
4,7347238,neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347238,0,7,0.764116,0,0
5,7347239,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347239,0,3,0.833017,0,0
6,7347699,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347699,1,0,0.899593,1,0
7,7347700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347700,0,0,0.685197,1,0
8,7347701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347701,0,3,0.877727,1,0
9,7347702,neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347702,1,8,0.629507,0,0


In [122]:
check_decision_direction_df = merged_df[merged_df['decision_direction'].isin([0,1])]
decision_direction_accuracy = check_decision_direction_df['correct_decision_direction'].mean()
decision_direction_accuracy

0.5070422535211268

In [133]:
check_decision_direction_high_acc_df = check_decision_direction_df[check_decision_direction_df['probability']>0.9]
decision_direction_accuracy_high_acc = check_decision_direction_high_acc_df['correct_decision_direction'].mean()
decision_direction_accuracy_high_acc

0.631578947368421

In [131]:
check_decision_direction_high_acc_df

,id,decision_direction,topic,topic_1,topic_2,topic_3,topic_code,topic_code1,topic_code2,topic_code3,opinion_id,predicted_label,predicted_topic_label,probability,correct_decision_direction,correct_topic_area
3,7347208,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347208,0,0,0.909268,1,0
12,7347705,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347705,1,9,0.916411,1,0
14,7347707,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347707,1,0,0.969903,1,0
22,7347716,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7347716,1,8,0.911027,1,0
30,2360385,1,first amendment; due process,first amendment,due process,None,NaN,2.0,3.0,NaN,2360385,1,1,0.925462,1,0
37,45264,1,civil rights,civil rights,None,None,1.0,1.0,NaN,NaN,45264,0,1,0.950116,0,1
53,9325303,1,criminal procedure,criminal procedure,None,None,0.0,0.0,NaN,NaN,9325303,0,0,0.901512,0,1
70,1153157,1,criminal procedure,criminal procedure,None,None,0.0,0.0,NaN,NaN,1153157,1,9,0.939549,1,0
71,6361550,1,economic activity,economic activity,None,None,7.0,7.0,NaN,NaN,6361550,0,7,0.964045,0,1
75,1392027,0,criminal procedure,criminal procedure,None,None,0.0,0.0,NaN,NaN,1392027,1,0,0.901234,0,1


In [123]:
check_topic_area_df = merged_df[
    (merged_df['topic_code'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])) |
    (merged_df['topic_code1'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])) |
    (merged_df['topic_code2'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])) |
    (merged_df['topic_code3'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]))
]
topic_area_accuracy = check_topic_area_df['correct_topic_area'].mean()
topic_area_accuracy

0.46153846153846156

In [124]:
check_topic_area_no_private_action_df = check_topic_area_df[
    (check_topic_area_df['topic_code'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])) |
    (check_topic_area_df['topic_code1'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])) |
    (check_topic_area_df['topic_code2'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])) |
    (check_topic_area_df['topic_code3'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]))
]                                                           
topic_area_accuracy_no_private_action = check_topic_area_no_private_action_df['correct_topic_area'].mean()
topic_area_accuracy_no_private_action

0.65625